1. Importação das Bibliotecas
   
Importa bibliotecas essenciais para análise de dados (pandas, numpy), visualização (matplotlib, seaborn) e aprendizado de máquina (scikit-learn).

In [1]:
pip install numpy


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split # Para validação cruzada
from sklearn.preprocessing import OneHotEncoder # Para codificação One-Hot

1. Carregamento e Preparação dos Dados:
- Carrega dados de treinamento e teste a partir de arquivos CSV.
- Remove a coluna 'PatientID', que provavelmente não é relevante para a previsão.
- Realiza uma análise exploratória de dados (EDA) para entender melhor os dados.

In [ ]:
# Carregando os dados
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
y_test = pd.read_csv('y_test.csv')

In [ ]:
# Removendo a coluna 'PatientID'
train.drop(['PatientID'], axis=1, inplace=True)
test.drop(['PatientID'], axis=1, inplace=True)

In [ ]:
train.dtypes

In [ ]:
train.head()

In [ ]:
# Somar os números de dados faltantes por parâmetro
train.isnull().sum().sort_values(ascending=False)

In [ ]:
train.describe()

1. Visualização dos Dados:

Utiliza bibliotecas de visualização para criar gráficos como histogramas, gráficos de barras, etc., para visualizar padrões nos dados.

In [ ]:
train.hist(figsize=(10,8));

In [ ]:
# Analisando a distribuição do colesterol pelo gênero
x = train[['Sex','Cholesterol']].groupby(['Sex']).count()
print(x) 

In [ ]:
# Analisando a distribuição do colesterol pela idade
x = train[['Age', 'Cholesterol']].groupby(['Age']).count()
print(x)

In [ ]:
# Analisando a distribuição do colesterol pelo gênero e idade
x = train[['Age', 'Sex', 'Cholesterol']].groupby(['Age','Sex']).count()
print(x)

In [ ]:
# Analisando a distribuição da Dieta pela idade e gênero
x = train[['Age', 'Sex', 'Diet']].groupby(['Age', 'Sex']).count()
print(x)

In [ ]:
# Analisando a distribuição do Nível de Estresse pela idade e gênero
x = train[['Age', 'Sex', 'StressLevel']].groupby(['Age', 'Sex']).count()
print(x)

In [ ]:
# Analisando a chance de risco de infarto pelo gênero
train[['Sex', 'HeartAttackRisk']].groupby(['Sex']).mean()

In [ ]:
# Plotar os gráficos para Heart Attack Risk vs Sex, Smoking, Diet e Previous Heart Problem
fig, (axis1, axis2, axis3, axis4) = plt.subplot(1,4,figsize=(12,4))

sns.barplot(x='Sex', y='HeartAttackRisk', data=train, ax=axis1)
sns.barplot(x='Smoking', y='HeartAttackRisk', data=train, ax=axis2)
sns.barplot(x='Diet', y='HeartAttackRisk', data=train, ax=axis3)
sns.barplot(x='PreviousHeartProblems', y='HertAttackRisk', data=train, ax=axis4);

In [ ]:
# FaceGrid pelos dados de 'Heart Attack Risk'  - Resultando dois gráficos com risco de ataque cardiaco , mas por idade
age_attack = sns.FacetGrid(train, col='HeartAttackRIsk')
age_attack.map(sns.histplot, 'Age')

In [ ]:
# FaceGrid pelos dados de 'Sex' - Resultando dois gráficos com masculino e feminino, mais por 'Heart Rate'
sex_rate = sns.FacetGrid(train, col='Sex')
sex_rate.map(sns.histplot, 'HeartRate')

In [ ]:
# FaceGrid pelos dados de 'Age' - Resultando de 10 gráficos representando os dez levels de Stress pela idade
age_stress = sns.FacetGrid(train, col='StressLevel')
age_stress.map(sns.histplot, 'Age')

In [ ]:
# FaceGrid pelos dados de 'Family History' -  Resultando dois gráficos com histórico familiar, mas por idade
age_familyhistory = sns.FacetGrid(train, col='FamilyHistory')
age_familyhistory.map(sns.histplot, 'Age')

In [ ]:
# Plotar uma scatter matrix com os atributos Idade, Sexo, Fumante, Colesterol e Nível de Estresse
columns = ['Age', 'Sex', 'Smoking', 'Cholesterol', 'StressLevel']
pd.plotting.scatter_matrix(train[columns], figsize=(15,10));

In [ ]:
# Plotar uma scatter matrix com os atributos de Idade, Colesterol e Nível de Estresse
columns = ['Age', 'Cholesterol', 'StressLevel']
pd.plotting.scatter_matrix(train[columns], figsize=(15, 10));

In [ ]:
# Plotar o heatmap para as variáveis numéricas
# Cálculo da correlação
print(train.head())
correlacao = train.corr(numeric_only=True)
print('\nCORRELAÇÃO: \N', correlacao)
# Plot gráfico Heatmap
sns.heatmap(correlacao, cmap='coolwarm', fmt='.2f', linewidths=0.1, vmax=1.0, square=True, linecolor='white', annot=True)

1. Codificação One-Hot:

Converte variáveis categóricas (por exemplo, sexo, tabagismo) em representações numéricas usando codificação one-hot para que possam ser usadas em modelos de aprendizado de máquina.

In [ ]:
# Codificação One-Hot para variáveis categóricas
encoder = OneHotEncoder(handle_unknown='ignore')
features_to_encode = ['Sex', 'Smoking', 'AlcoholConsumption', 'Diet', 'PreviousHeartProblems', 'MedicationUse']
encoder.fit(train[features_to_encode])

In [ ]:
# Transformar os dados de treinamento e teste
train_encoded = encoder.transform(train[features_to_encode]).toarray()
test_encoded = encoder.transform(test[features_to_encode]).toarray()

In [ ]:
# Renomear as colunas codificadas
new_column_names = encoder.get_feature_names_out(features_to_encode)
train_encoded = pd.DataFrame(train_encoded, columns=new_column_names)
test_encoded = pd.DataFrame(test_encoded, columns=new_column_names)

In [ ]:
# Concatenar os dados codificados aos DataFrames  originais
train = pd.concat([train, train_encoded], axis=1)
test = pd.concat([test, test_encoded], axis=1)

In [ ]:
# Removendo as colunas originais após a codificação
train.drop(features_to_encode, axis=1, inplace=True)
test.drop(features_to_encode, axis=1, inplace=True)

In [ ]:
# Separando o alvo dos dados de treinamento
target = train['HeartAttackRisk']
train.drop(['HeartAttackRisk'], axis=1, inplace=True)

1. Imputação de Valores:

Trata valores ausentes nos dados substituindo-os pela mediana dos valores da coluna.

In [ ]:
# Imputação de valores ausentes com a mediana
for column in train.columns:
    train[column].fillna(train[column].median(), inplace=True)
    test[column].fillna(train[column].median(), inplace=True)

1. Divisão e Treinamento do Modelo:

- Divide os dados de treinamento em conjuntos de treinamentos e validação para avaliar o desempenho do modelo.
- Cria e treina um modelo de Árvore de Decisão para prever o risco de ataque cardíaco.

In [ ]:
# Dividindo os dados de treinamento para validação cruzada
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=42)

In [ ]:
# Converter nomes de colunas para string
X_train.columns = X_train.columns.astype(str)
X_val.columns = X_val.columns.astype(str)

In [ ]:
# Criando e treinando o modelo de árvore de decisão
tree_model = DecisionTreeClassifier(max_depth=9, criterion='entropy')
tree_model.fit(X_train, y_train)

In [ ]:
# Avaliando o modelo no conjunto de validação
y_pred_tree_val = tree_model.predict(X_val)
accuracy_val = accuracy_score(y_val, y_pred_tree_val)
print('Acurácia no conjunto de validação: ', accuracy_val)

In [ ]:
# Remover a coluna 'HeartAttackRisk' do DataFrame test
test.drop(['HeartAttackRisk'], axis=1, inplace=True)

1. Avaliação e Previsões do Modelo:

- Avalia o desempenho do modelo nos conjuntos de validação e teste usando a métrica de acurácia.
- Faz previsões no conjunto de teste usando o modelo treinado.

In [ ]:
# Fazendo previsões no conjunto de teste
y_pred_tree = tree_model.predict(test)
accuracy_test = accuracy_score(np.array(y_test['HeartAttackRisk']), y_pred_tree)
print('Acurácia no conjunto de teste: ', accuracy_test)

In [ ]:
# Plotando a árvore de decisão
fig = plt.figure(figsize=(15, 10))
_ = tree.plot_tree(tree_model, feature_names=train.columns, class_names={0: 'Não', 1: 'Sim'})

1. Função de Previsão:

Define uma função para receber informações do paciente, realizar as transformações necessárias nos dados e fornecer uma previsão de risco de ataque cardíaco usando o modelo treinado.

In [ ]:
def prever_risco():
    sexo = int(input("Digite o sexo (0: Feminino, 1: Masculino): "))
    idade = int(input("Digite a idade: "))
    fuma = int(input("Fuma? (0: Não, 1: Sim): "))
    obesidade = int(input("Obeso? (0: Não, 1: Sim): "))
    alcool = int(input("Consome Alcool? (0: Não, 1: Sim): "))
    diabetes = int(input("Diabético? (0: Não, 1: Sim): "))
    dieta = int(input("Dieta (0: Saudável, 1: Não Saudável, 2: Moderada): "))
    probemas_cardiacos = int(input("Problemas cardíacos prévios? (0: Não, 1: Sim): "))
    uso_medicacao = int(input("Uso de medicação? (0: Não, 1: Sim): "))
    colesterol = int(input("Taxa de Colesterol: "))
    # Criando um DataFrame com os dados do paciente
    dados_entrada = {'Age':[idade],
                     'Cholesterol':[colesterol], #[train['Cholesterol'].median()], #Valor da mediana do treino
                     'HeartRate':[train['HeartRate'].median()], #Valor da mediana do treino
                     'Diabetes':[diabetes],
                     'FamilyHistory':[train['FamilyHistory'].median()], #Valor da mediana do treino
                     'Obesity':[obesidade],
                     'ExerciseHoursPerWeek':[train['ExerciseHoursPerWeek'].median()], #Valor da mediana do treino
                     'StressLevel':[train['StressLecel'].median()], #Valor da mediana do treino
                     'SedentaryHoursPerDay':[train['SedentaryHoursPerDay'].median()], #Valor da mediana do treino
                     'BMI':[train['BMI'].median()], #Valor da mediana do treino
                     'Triglycerides':[train['Triglycerides'].median()], #Valor da mediana do treino
                     'PhysicalActivityDaysPerWeek':[train['PhysicalActivityDaysPerWeek'].median()], # Valor da mediana do treino
                     'SleepHoursPerDay':[train['SleepHoursPerDay'].median()], # Valor da mediana do treino
                     'Sex':[sexo],
                     'Smoking':[fuma],
                     'AlcoholConsumption':[alcool],
                     'Diet':[dieta],
                     'MedicationUse':[uso_medicacao]}
    pessoa = pd.DataFrame(dados_entrada)

    # Codificando as variáveis categóricas
    pessoa_encoded = encoder.transform(pessoa[features_to_encode]).toarray()
    new_column_names = encoder.get_feature_names_out(features_to_encode)
    pessoa_encoded = pd.DataFrame(pessoa_encoded, columns=new_column_names)

    # Concatenar os dados codificados ao DataFrame original
    pessoa = pd.concat([pessoa, pessoa_encoded], axis=1)

    # Remover as colunas originais após a codificação
    pessoa.drop(features_to_encode, axis=1, inplace=True)

    # Converter nomes de colunas para string
    pessoa.columns = pessoa.columns.astype(str)

    # Fazendo a previsão
    resultado = tree_model.predict(pessoa)

    if resultado == 0:
        print("\nPaciente sem risco de infarto.")
    else:
        print("\nPaciente com risco de infarto.")

prever_risco()